In [1]:
%load_ext lab_black

In [2]:
import pickle
import pandas as pd
from collections import defaultdict
import re
import numpy as np
import os
import matplotlib.pyplot as plt
import datetime

In [4]:
files = [
    "scraped_financials2.csv",
    "scraped_financials.csv",
    "scraped_financials31.csv",
    "scraped_financials32.csv",
    "scraped_financials33.csv",
    "scraped_financials34.csv",
    "scraped_financials1631271189650785000.csv",
]

dfs = [
    pd.read_csv(os.path.join("../data/intermediate", file), index_col=0)
    for file in files
]

companies = pd.concat(dfs).reset_index(drop=True)

In [5]:
companies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7391 entries, 0 to 7390
Data columns (total 5 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   total_current_assets       3411 non-null   float64
 1   total_current_liabilities  3362 non-null   float64
 2   CIK                        7391 non-null   int64  
 3   date_filled                7391 non-null   object 
 4   url                        7391 non-null   object 
dtypes: float64(2), int64(1), object(2)
memory usage: 288.8+ KB


In [6]:
companies.head()

,total_current_assets,total_current_liabilities,CIK,date_filled,url
0,486380.0,185896.0,816159,2016-12-31,https://www.sec.gov/Archives/edgar/data/000081...
1,473359.0,149664.0,816159,2015-12-31,https://www.sec.gov/Archives/edgar/data/000081...
2,399841.0,180267.0,816159,2014-12-31,https://www.sec.gov/Archives/edgar/data/000081...
3,340550.0,162318.0,816159,2013-12-31,https://www.sec.gov/Archives/edgar/data/000081...
4,306805.0,143452.0,816159,2012-12-31,https://www.sec.gov/Archives/edgar/data/000081...


In [7]:
len(companies["CIK"].unique())

552

In [8]:
# FIXME some companies have duplicate entries for date_filled. Amendments?
print(companies.shape)
companies.drop_duplicates(subset=["CIK", "date_filled"], inplace=True)
companies.shape

(7391, 5)


(6903, 5)

In [9]:
companies["date_filled"] = pd.to_datetime(companies["date_filled"])
companies["CIK"] = companies["CIK"].astype("category")
companies["current_ratio"] = (
    companies["total_current_assets"] / companies["total_current_liabilities"]
)

In [10]:
pd.set_option("max_colwidth", 400)


def get_fiscal_year(dt):
    if dt.month < 6:
        return dt.year - 1
    else:
        return dt.year


companies["fiscal_year"] = companies["date_filled"].apply(lambda x: get_fiscal_year(x))
companies[
    ["total_current_assets", "total_current_liabilities", "url", "fiscal_year"]
].head()

,total_current_assets,total_current_liabilities,url,fiscal_year
0,486380.0,185896.0,https://www.sec.gov/Archives/edgar/data/0000816159/000157104917001894/t1700128_10k.htm,2016
1,473359.0,149664.0,https://www.sec.gov/Archives/edgar/data/0000816159/000157104916012144/t1600089_10k.htm,2015
2,399841.0,180267.0,https://www.sec.gov/Archives/edgar/data/0000816159/000157104915001464/t81389_10k.htm,2014
3,340550.0,162318.0,https://www.sec.gov/Archives/edgar/data/0000816159/000118811214000551/t78398_10k.htm,2013
4,306805.0,143452.0,https://www.sec.gov/Archives/edgar/data/0000816159/000114036113009925/form10k.htm,2012


In [11]:
# save processed company dataset
companies.to_csv("../data/processed/scraped_companies_added.csv")